In [5]:
# Create sqlite database
import sqlite3

conn = sqlite3.connect('experiments.db')
c = conn.cursor()
# c.execute('''CREATE TABLE experiments (code text, commit_message text, f

In [9]:
import openai
import random
import os
import json
import git

repo = git.Repo(".")

# Get staged changes
staged = repo.head.commit.diff(None, create_patch=True, cached=True)

prompt = ""
for i, code_row in enumerate(staged):
    prompt += f" file path A: {code_row.a_path} | file path B: {code_row.b_path}\n\n"
    prompt += code_row.diff.decode("utf-8")
    prompt += "\n---\n" if i < len(code_row.diff) - 1 else ""
prompt += "\n\n###\n\n"


openai.api_key = "<KEY>"
response = openai.Completion.create(
    model="curie:ft-personal:commit-2023-01-07-01-35-45",
    prompt=prompt,
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=["\n"]
)
print(response["choices"][0]["text"])


InvalidRequestError: This model's maximum context length is 2049 tokens, however you requested 3378 tokens (3122 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.

In [1]:
%pip install --upgrade openai
%pip install GitPython

distutils: /Users/qasimkhawaja/miniforge3/include/python3.9/UNKNOWN
sysconfig: /Users/qasimkhawaja/miniforge3/include/python3.9
user = False
home = None
root = None
prefix = None
distutils: /Users/qasimkhawaja/miniforge3/include/python3.9/UNKNOWN
sysconfig: /Users/qasimkhawaja/miniforge3/include/python3.9
user = False
home = None
root = None
prefix = None
Note: you may need to restart the kernel to use updated packages.
distutils: /Users/qasimkhawaja/miniforge3/include/python3.9/UNKNOWN
sysconfig: /Users/qasimkhawaja/miniforge3/include/python3.9
user = False
home = None
root = None
prefix = None
distutils: /Users/qasimkhawaja/miniforge3/include/python3.9/UNKNOWN
sysconfig: /Users/qasimkhawaja/miniforge3/include/python3.9
user = False
home = None
root = None
prefix = None
Note: you may need to restart the kernel to use updated packages.


In [17]:
import sqlite3
import git
import re
def create_db(name):
    conn = sqlite3.connect(name)
    c = conn.cursor()
    c.execute("CREATE TABLE experiments (code text, commit_message text, file_a text NULL, file_b text NULL)")

def batch_insert(name, codes, commit_messages, files_a, files_b):
    conn = sqlite3.connect(name)
    c = conn.cursor()
    c.executemany("INSERT INTO experiments VALUES (?,?,?,?)", zip(codes, commit_messages, files_a, files_b))
    conn.commit()
    
def scrape_repo(repo_path):
    repo = git.Repo(repo_path)
    codes = []
    commit_messages = []
    files_a = []
    files_b = []
    count = 0
    for commit in repo.iter_commits():
        try:
            if not commit.parents:
                continue
            code_diff = commit.parents[0].diff(commit, create_patch=True)
            total_len = sum([len(diff.diff.decode("utf-8")) for diff in code_diff])
        except:
            print(f"Skipping commit {commit.hexsha}")
            continue
            
        count += 1
        
        if count % 100 == 0:
            print(f"Processed {count} commits")
        if count > 1000:
            print("Breaking")
            break
        
        if total_len / 4 > 4000:
            print(f"Skipping commit {commit.hexsha} because it is too long")
            continue
        regex = r"^(build|chore|ci|docs|feat|fix|perf|refactor|revert|style|test|tests)(\(.*\))?: .*"
        if not re.match(regex, commit.message.split("\n")[0]):
            print(f"Skipping commit {commit.hexsha} because it does not match regex")
            print(commit.message.split("\n")[0])
            continue
        for diff in code_diff:
            codes.append(diff.diff.decode("utf-8"))
            commit_messages.append(commit.message)
            files_a.append(diff.a_path)
            files_b.append(diff.b_path)
        
    return codes, commit_messages, files_a, files_b

REPOS = [
    # "https://github.com/bitcoin/bitcoin.git",
    # "https://github.com/tpope/vint.git",
    # "https://github.com/jina-ai/jina.git",
    # "https://github.com/typescript-eslint/typescript-eslint.git"
    # "https://github.com/axios/axios.git",
    # "https://github.com/angular/angular.git",
    # "https://github.com/conventional-commits/conventionalcommits.org.git",
    # "https://github.com/conventional-commits/parser.git",
    # "https://github.com/tomasbjerre/git-changelog-lib.git",
    "https://github.com/vuejs/vue.git"
]
def main():
    for repo in REPOS:
        repo_name = repo.split("/")[-1].split(".git")[0]
        print(f"Cloning {repo_name}")
        os.system(f"git clone {repo}")
        data = scrape_repo(repo_name)
        create_db(f"{repo_name}.db")
        batch_insert(f"{repo_name}.db", *data)
    


In [23]:
main()

Cloning vue
Skipping commit 9dd006b481b4299462e044741bac0861c0b1775c because it does not match regex
release: v2.7.14
Skipping commit a7a71ad99071c8c48ae9c106a19f81e70ded4898 because it does not match regex
Update the dev to main in the PR template (#12834)
Skipping commit df281b0e4c400be18df8bcf4e83df8b25a81e21b because it does not match regex
release: v2.7.13
Skipping commit 4cad41007174a7eeebf4472ddbd7b3b693db03f1 because it does not match regex
release: v2.7.12
Skipping commit e80cd09fff570df57d608f8f5aaccee6d7f31917 because it does not match regex
Revert "fix(setup): setup hook should be called before beforeCreate"
Skipping commit c61395d3f1ed926d2783d8460970ed20ea01dc4f because it does not match regex
release: v2.7.11
Skipping commit cc14d4452c66e90341b7f9a5cdd2b7acffa529e9 because it is too long
Skipping commit ee57d9fd1d51abe245c6c37e6f8f2d45977b929e because it does not match regex
release: v2.7.10
Skipping commit 15618888cb6aae2b6f0151b32c261b1fc19db1b8 because it does not mat

OperationalError: table experiments already exists

In [24]:
import json
def process_db(name):
    conn = sqlite3.connect(name)
    c = conn.cursor()
    c.execute("SELECT DISTINCT commit_message FROM experiments")
    commit_messages = c.fetchall()
    print(f"Found {len(commit_messages)} unique commit messages")
    data = []
    for commit_message in commit_messages:
        c.execute("SELECT code, file_a, file_b FROM experiments WHERE commit_message=?", (commit_message[0],))
        code_rows = c.fetchall()
        prompt = ""
        for i, code_row in enumerate(code_rows):
            code = code_row[0]
            file_a = code_row[1]
            file_b = code_row[2]
            prompt += f" file path A: {file_a} | file path B: {file_b}\n\n"
            prompt += code
            if i < len(code_rows) - 1:
                prompt += "\n---\n"
        prompt += "\n\n###\n\n"
        completion = commit_message[0].split("\n")[0]
        # Replace (#number) with (#<issue-num>)
        completion = re.sub(r"\(#\d+\)", "(#<issue-num>)", completion)
        obj = {"prompt": prompt, "completion": completion}
        data.append(obj)
    if not os.path.exists(f"{name}.jsonl"):
        os.system(f"rm {name}.jsonl")
    with open(f"{name}.jsonl", "w") as f:
        for d in data:
            f.write(json.dumps(d) + "\n")
    
            
        
            
    
def bulk_process():
    for repo in REPOS:
        repo_name = "dbs/"+repo.split("/")[-1].split(".git")[0]
        print(f"Processing {repo_name}")
        process_db(f"{repo_name}.db")
    # Merge all jsonl files into one
    # os.system("cat dbs/*.jsonl > all.jsonl")

In [25]:
print("Processing databases")
bulk_process()


Processing databases
Processing dbs/vue
Found 750 unique commit messages


In [5]:
os.system("cat dbs/*.jsonl > all.jsonl")

0

In [2]:
import random
import openai
import json
from numpy import dot
def chart_similarity(name):
    # Load data
    openai.api_key = "<KEY>"
    with open(name, "r") as f:
        data = f.readlines()
        # Pick a random sample of 10
        data = random.sample(data, 10)
        data = [json.loads(d) for d in data]
    # Use openai to get similarity scores for each prompt
    similarity_scores = []
    for d in data:
        try:
            prompt = d["prompt"]
            original_completion = d["completion"]
            # calculate embedding for original completion and generated completion
            original_completion_embedding = openai.Embedding.create(input=original_completion, model="text-embedding-ada-002")
            generated_completion = openai.Completion.create(engine="curie:ft-personal:commit-gen-2023-01-07-23-56-35", prompt=prompt, temperature=0.8, max_tokens=50, top_p=1, frequency_penalty=0, presence_penalty=0, stop=["\n"])
            generated_completion_embedding = openai.Embedding.create(input=generated_completion["choices"][0]["text"], model="text-embedding-ada-002")
            generated_completion_2 = openai.Completion.create(engine="curie:ft-personal:commit-2023-01-07-01-35-45", prompt=prompt, temperature=0.8, max_tokens=50, top_p=1, frequency_penalty=0, presence_penalty=0, stop=["\n"])
            generated_completion_embedding_2 = openai.Embedding.create(input=generated_completion_2["choices"][0]["text"], model="text-embedding-ada-002")
            print(f"Similarity score: {dot(original_completion_embedding['data'][0]['embedding'], generated_completion_embedding['data'][0]['embedding'])}")
            print(f"Original commit message: {original_completion.strip()}")
            print(f"Generated commit message: {generated_completion['choices'][0]['text']}\n")
            print(f"Similarity score 2: {dot(original_completion_embedding['data'][0]['embedding'], generated_completion_embedding_2['data'][0]['embedding'])}")
            print(f"Original commit message 2: {original_completion.strip()}")
            print(f"Generated commit message 2: {generated_completion_2['choices'][0]['text']}\n")
        except Exception as e:
            # print(e)
            continue
chart_similarity("dbs/vue.db_prepared.jsonl")
        
        

Similarity score: 0.8452262273507115
Original commit message: feat(warns): avoid warning native modifiers on dynamic components (#<issue-num>)
Generated commit message:  fix(core): Warn if native modifier is used on a dynamic component (#<issue-num>)

Similarity score 2: 0.6271416738734499
Original commit message 2: feat(warns): avoid warning native modifiers on dynamic components (#<issue-num>)
Generated commit message 2: 

Similarity score: 0.8076614368595502
Original commit message: feat(core): expose all slots on $scopedSlots as functions
Generated commit message:  fix(instance): render functions should be able to have access to $scopedSlots (#<issue-num>)

Similarity score 2: 0.5916308864582759
Original commit message 2: feat(core): expose all slots on $scopedSlots as functions
Generated commit message 2: 

Similarity score: 0.8476781645748372
Original commit message: test: add a test case for ssr max stack size limit
Generated commit message:  test(docs-infra): add test for handl